In [2]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.animation as animation
import matplotlib as mpl
from projet_6_bis_iteration_friendly import *

# --- Paramètres du domaine et numérique ---
nx, ny = 31, 31
Lx, Ly = 1.0, 1.0
dt = 1e-4  # à ajuster si ça diverge à grand Re/Ra

# --- Grilles de paramètres ---
U0_list = [0.01, 0.05, 0.1, 0.2]              # -> plusieurs Re
Ra_list = [1e3, 1e4, 1e5, 1e6]                # -> plusieurs Ra

Nu_map = np.zeros((len(U0_list), len(Ra_list)))
Re_list = np.zeros(len(U0_list))

for i, U0 in enumerate(U0_list):
    for j, Ra in enumerate(Ra_list):
        Nu, Re = global_resolution(nx, ny, Lx, Ly, dt, U0, Ra)
        Nu_map[i, j] = Nu
        Re_list[i] = Re
        print(f"U0={U0:.3g} | Re={Re:.3e} | Ra={Ra:.1e} -> Nu={Nu:.4f}")


Le nombre de Courant est égale à : 3.0000000000000004e-05
Re = 999.9999999999999
Itération 0: Résidu w = inf, Résidu T = 1.23e-04
Itération 10: Résidu w = 9.99e-02, Résidu T = 1.22e-04
Itération 20: Résidu w = 4.99e-02, Résidu T = 1.22e-04


c:\Users\33672\Documents\Cours\Transfert num vf\Thermal-transfert\projet_6_bis_iteration_friendly.py:62: RuntimeWarning: divide by zero encountered in scalar divide
  res_w.append(np.linalg.norm(w_new - w)/np.linalg.norm(w))


Itération 30: Résidu w = 3.32e-02, Résidu T = 1.22e-04
Itération 40: Résidu w = 2.49e-02, Résidu T = 1.21e-04
Itération 50: Résidu w = 1.99e-02, Résidu T = 1.21e-04
Itération 60: Résidu w = 1.65e-02, Résidu T = 1.21e-04
Itération 70: Résidu w = 1.41e-02, Résidu T = 1.20e-04
Itération 80: Résidu w = 1.24e-02, Résidu T = 1.20e-04
Itération 90: Résidu w = 1.10e-02, Résidu T = 1.20e-04
Itération 100: Résidu w = 9.86e-03, Résidu T = 1.19e-04
Itération 110: Résidu w = 8.96e-03, Résidu T = 1.19e-04
Itération 120: Résidu w = 8.20e-03, Résidu T = 1.19e-04
Itération 130: Résidu w = 7.56e-03, Résidu T = 1.18e-04
Itération 140: Résidu w = 7.01e-03, Résidu T = 1.18e-04
Itération 150: Résidu w = 6.53e-03, Résidu T = 1.18e-04
Itération 160: Résidu w = 6.12e-03, Résidu T = 1.18e-04
Itération 170: Résidu w = 5.75e-03, Résidu T = 1.17e-04
Itération 180: Résidu w = 5.42e-03, Résidu T = 1.17e-04
Itération 190: Résidu w = 5.13e-03, Résidu T = 1.17e-04
Itération 200: Résidu w = 4.87e-03, Résidu T = 1.16e-04

KeyboardInterrupt: 

In [ ]:
plt.figure()
# imshow a besoin d'un tableau 2D; on met Ra en x, Re en y
im = plt.imshow(
    Nu_map,
    aspect="auto",
    origin="lower",
    extent=[np.log10(Ra_list[0]), np.log10(Ra_list[-1]), np.log10(Re_list[0]), np.log10(Re_list[-1])]
)
plt.colorbar(im, label="Nusselt moyen (Nu)")
plt.xlabel("log10(Ra)")
plt.ylabel("log10(Re)")
plt.title("Carte Nu(Re, Ra)")
plt.show()


In [ ]:
plt.figure()
for i, Re in enumerate(Re_list):
    plt.plot(Ra_list, Nu_map[i, :], marker="o", label=f"Re={Re:.2e}")

plt.xscale("log")
plt.xlabel("Rayleigh (Ra)")
plt.ylabel("Nusselt moyen (Nu)")
plt.title("Nu = f(Ra) pour différents Re")
plt.grid(True, which="both", linestyle="--", alpha=0.4)
plt.legend()
plt.show()
